In [1]:
import os
from PIL import Image
import numpy as np
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A
import imgaug as ia
import imgaug.augmenters as iaa

In [2]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 18
    
    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(8, 8))

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(8, 8))

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original image', fontsize=fontsize)
        
        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original mask', fontsize=fontsize)
        
        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed image', fontsize=fontsize)
        
        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed mask', fontsize=fontsize)

In [1]:
# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.


# def sometimes_1(aug): return iaa.Sometimes(0.6, aug)
# def sometimes_2(aug): return iaa.Sometimes(0.2, aug)


# image = np.array(Image.open(
#     '/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/1-Bipolar_Forceps/image_pos/seq_4_frame003.png'), dtype=np.uint8)
# segmap = np.array(Image.open(
#     '/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/1-Bipolar_Forceps/mask_pos/seq_4_frame003.png'), dtype=np.int32)
# images = np.expand_dims(image, axis=0)
# segmaps = np.expand_dims(segmap, axis=0)
# segmaps = np.expand_dims(segmaps, axis=3)

# seq = iaa.Sequential([
#     # apply the following augmenters to most images
#     iaa.Fliplr(0.2),  # horizontally flip 50% of all images
#     iaa.Flipud(0.2),  # vertically flip 20% of all images
#     sometimes_2(iaa.Crop(px=(10, 200))),

#     sometimes_1(iaa.Affine(
#         # scale images to 80-120% of their size, individually per axis
#         scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
#         # translate by -20 to +20 percent (per axis)
#         translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
#         rotate=(-45, 45),  # rotate by -45 to +45 degrees
#         # shear=(-16, 16), # shear by -16 to +16 degrees
#         order=[0, 1],  # use nearest neighbour or bilinear interpolation (fast)
#         # cval=(0, 255), # if mode is constant, use a cval between 0 and 255
#         cval=0,
#         # use any of scikit-image's warping modes (see 2nd image from the top for examples)
#         mode='constant'
#     )),
#     # improve or worsen the contrast
#     # sometimes_1(iaa.LinearContrast((0.5, 2.0), per_channel=0.5)),
#     # sometimes_2(iaa.PerspectiveTransform(scale=(0.1, 0.2))),

#     sometimes_1(iaa.OneOf([
#         # blur images with a sigma between 0 and 3.0
#         iaa.GaussianBlur((0, 3.0)),
#         # blur image using local means with kernel sizes between 2 and 7
#         iaa.AverageBlur(k=(2, 7)),
#         # blur image using local medians with kernel sizes between 2 and 7
#         iaa.MedianBlur(k=(3, 11)),
#     ])),

# ],
#     random_order=True)

In [3]:
def sometimes_1(aug): return iaa.Sometimes(0.6, aug)
def sometimes_2(aug): return iaa.Sometimes(0.2, aug)
def sometimes_3(aug): return iaa.Sometimes(0.4, aug)
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontally flip 50% of all images
    iaa.Flipud(0.2),  # vertically flip 20% of all images
    sometimes_2(iaa.Crop(px=(10, 200))),
    # improve or worsen the contrast
    # sometimes_1(iaa.LinearContrast((0.5, 2.0))), # +
    # sometimes_1(iaa.LinearContrast((0.5, 2.0), per_channel=0.5)),
    sometimes_2(iaa.PerspectiveTransform(scale=(0.1, 0.2))), # +
    sometimes_1(iaa.Affine(
        # scale images to 80-120% of their size, individually per axis
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        # translate by -20 to +20 percent (per axis)
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        rotate=(-45, 45),  # rotate by -45 to +45 degrees
        # shear=(-16, 16), # shear by -16 to +16 degrees
        order=[0, 1],  # use nearest neighbour or bilinear interpolation (fast)
        # cval=(0, 255), # if mode is constant, use a cval between 0 and 255
        cval=0,
        # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        mode='constant'
    )),
    # sometimes_1(iaa.OneOf([
    #         # iaa.Multiply((0.5, 1.5), per_channel=0.5),
    #         iaa.Multiply((0.5, 1.5)),
    #         iaa.FrequencyNoiseAlpha(
    #             exponent=(-4, 0),
    #             # first=iaa.Multiply((0.5, 1.5), per_channel=True),
    #             first=iaa.Multiply((0.5, 1.5)),
    #             second=iaa.LinearContrast((0.5, 2.0))
    #         )
    #     ])), # +
    # iaa.Multiply((0.5, 1.5), per_channel=0.5), # +
    # iaa.SimplexNoiseAlpha(iaa.OneOf([
    #                 iaa.EdgeDetect(alpha=(0, 0.5)),
    #                 iaa.DirectedEdgeDetect(alpha=(0, 0.5), direction=(0.0, 1.0)),
    #             ])),
    # sometimes_1(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)

    sometimes_3(iaa.OneOf([
        # blur images with a sigma between 0 and 3.0
        iaa.GaussianBlur((0, 3.0)),
        # blur image using local means with kernel sizes between 2 and 7
        iaa.AverageBlur(k=(2, 7)),
        # blur image using local medians with kernel sizes between 2 and 7
        iaa.MedianBlur(k=(3, 11)),
    ])),
],random_order=True)

In [1103]:
# 2 base image per instru
from tqdm import tqdm
import sys
N = 50 # number of variants for each image/mask
content_list = os.listdir()
folder_list = []
for i in content_list:
    if os.path.isdir(i):
        folder_list.append(i)
print(folder_list) # ['3-Large_Needle_Driver', '5-Ultrasound_Probe', '6-Suction_Instrument', '1-Bipolar_Forceps', '4-Monopolar_Curved_Scissors', '7-Clip_Applier', '2-Prograsp_Forceps']
for folder in folder_list[:]:
    image_list = os.listdir(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/', folder, 'image_2'))
    # print(image_list)
    for img_name in image_list:
        # image = np.array(Image.open(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/', folder, 'image_pos', img_name)), dtype=np.uint8)
        image = np.array(Image.open(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/', folder, 'image_2', img_name)).convert('RGB'), dtype=np.uint8)
        # print(image.shape)
        # sys.exit(0)
        segmap = np.array(Image.open(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/', folder, 'mask_pos', img_name)), dtype=np.int32)
        images = np.expand_dims(image, axis=0)
        segmaps = np.expand_dims(segmap, axis=0)
        segmaps = np.expand_dims(segmaps, axis=3)

        for i in tqdm(range(N), ncols=100, desc=folder+'-'+img_name):
            images_aug, segmaps_aug = seq(images=images, segmentation_maps=segmaps)
            images_aug = np.squeeze(images_aug)
            segmaps_aug = np.squeeze(segmaps_aug)
            segmaps_aug = segmaps_aug.astype('uint8')
            
            image_pil = Image.fromarray(images_aug).convert('RGBA')
            mask_pil = Image.fromarray(segmaps_aug).convert('RGBA')

            image_np = np.array(image_pil)
            mask_np = np.array(mask_pil)

            image_np[mask_np[:,:,:]==0] = 255
            img = Image.fromarray(image_np)
            datas = img.getdata()
            newData = []
            for items in datas:
                if items[0] == 255 and items[1] == 255 and items[2] == 255:
                    newData.append((255, 255, 255, 0))
                else:
                    newData.append(items)
            img.putdata(newData)
            # img.save(os.path.join(folder_list[i], 'image_pos', image_name), "PNG")

            # print(image_np.shape, mask_np.shape)
            # visualize(img, mask_pil)
            # sys.exit(0)
            # print(np.unique(segmaps_aug))
            # img = Image.fromarray(images_aug)
            msk = Image.fromarray(segmaps_aug)
            # visualize(img, msk)
            img.save('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/'+ folder + '/image_2_50/' + img_name[:-4] + '_' + str(i)+'.png', 'PNG')
            msk.save('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/'+ folder + '/mask_2_50/' + img_name[:-4] + '_' + str(i)+'.png', 'PNG')

['3-Large_Needle_Driver', '5-Ultrasound_Probe', '6-Suction_Instrument', '1-Bipolar_Forceps', '4-Monopolar_Curved_Scissors', '7-Clip_Applier', '2-Prograsp_Forceps']


2-Prograsp_Forceps-seq_1_frame017.png: 100%|████████████████████████| 50/50 [00:29<00:00,  1.69it/s]


In [4]:
# 3 base image per instru
from tqdm import tqdm
import sys
N = 50 # number of variants for each image/mask
content_list = os.listdir()
folder_list = []
for i in content_list:
    if os.path.isdir(i):
        folder_list.append(i)
print(folder_list) # ['3-Large_Needle_Driver', '5-Ultrasound_Probe', '6-Suction_Instrument', '1-Bipolar_Forceps', '4-Monopolar_Curved_Scissors', '7-Clip_Applier', '2-Prograsp_Forceps']
for folder in folder_list[:]:
    image_list = os.listdir(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/', folder, 'image_3'))
    # print(image_list)
    for img_name in image_list:
        # image = np.array(Image.open(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/', folder, 'image_pos', img_name)), dtype=np.uint8)
        image = np.array(Image.open(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/', folder, 'image_3', img_name)).convert('RGB'), dtype=np.uint8)
        # print(image.shape)
        # sys.exit(0)
        segmap = np.array(Image.open(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/', folder, 'mask_pos_3', img_name)), dtype=np.int32)
        images = np.expand_dims(image, axis=0)
        segmaps = np.expand_dims(segmap, axis=0)
        segmaps = np.expand_dims(segmaps, axis=3)

        for i in tqdm(range(N), ncols=100, desc=folder+'-'+img_name):
            images_aug, segmaps_aug = seq(images=images, segmentation_maps=segmaps)
            images_aug = np.squeeze(images_aug)
            segmaps_aug = np.squeeze(segmaps_aug)
            segmaps_aug = segmaps_aug.astype('uint8')
            
            image_pil = Image.fromarray(images_aug).convert('RGBA')
            mask_pil = Image.fromarray(segmaps_aug).convert('RGBA')

            image_np = np.array(image_pil)
            mask_np = np.array(mask_pil)
            
            # make transparent
            image_np[mask_np[:,:,:]==0] = 255
            img = Image.fromarray(image_np)
            datas = img.getdata()
            newData = []
            for items in datas:
                if items[0] == 255 and items[1] == 255 and items[2] == 255:
                    newData.append((255, 255, 255, 0))
                else:
                    newData.append(items)
            img.putdata(newData)
            # img.save(os.path.join(folder_list[i], 'image_pos', image_name), "PNG")

            # print(image_np.shape, mask_np.shape)
            # visualize(img, mask_pil)
            # sys.exit(0)
            # print(np.unique(segmaps_aug))
            # img = Image.fromarray(images_aug)
            msk = Image.fromarray(segmaps_aug)
            # visualize(img, msk)
            img.save('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/'+ folder + '/image_3_50/' + img_name[:-4] + '_' + str(i)+'.png', 'PNG')
            msk.save('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/'+ folder + '/mask_3_50/' + img_name[:-4] + '_' + str(i)+'.png', 'PNG')

['3-Large_Needle_Driver', '5-Ultrasound_Probe', '6-Suction_Instrument', '1-Bipolar_Forceps', '4-Monopolar_Curved_Scissors', '7-Clip_Applier', '2-Prograsp_Forceps']


2-Prograsp_Forceps-seq_14_frame028.png: 100%|███████████████████████| 50/50 [00:30<00:00,  1.64it/s]
